In [1]:
import numpy as np
import pandas as pd
import os
from pandas_plink import read_plink
import pandas_plink as pdp
from pandas_plink import get_data_folder
import random
import json
from IPython.display import clear_output
import xarray as xr


In [2]:
print("done")

done


In [3]:
# select variants randomly
# select variants by rarity
# use linear calculation
# use non linear calculation
# with/without PC
# how rare should the phenotype be - like an existing one
# create syntetic covariates

In [41]:
GENETICS_PATH = "/sci/archive/michall/roeizucker/small_genetics"
COVARS_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK"
PLINK_PATH = "/sci/nosnap/michall/roeizucker/plink2"
TEST_COVAR = "covariates_sex_year_of_birth_AC_batch_40_PC.txt"
# TEST_CHROM = "/sci/archive/michall/roeizucker/small_genetics/allchr_thin_12"
TEST_CHROM = "allchr_thin_fml"
CHROM_PREFIX = "chr"
CHROM_SUFFIX = "_thin00005"
PLINK_RESULT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results/chr22_very_reduced_simulation"
PLINK_RESULT_DIR = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results/"
PHENOTYPE_FILE_RESULT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/phenotypes/generated_phen.txt"
PERSON_BATCH_SIZE = 2000
VERBOSE = False

FILE_ENDING = "PHENO1.glm.linear"

VERSION = "multi_effect_score_0.7"
COVAR_DF_PREFIX = "temp_covar_df"
INCLUDED_COVARS = ["sex","year_of_birth","AC","batch"]
AC_VALS = ['AC_stockport_pilot','AC_manchester','AC_oxford','AC_cardiff','AC_glasgow','AC_edinburgh','AC_stoke','AC_reading','AC_bury','AC_newcastle','AC_bristol','AC_barts','AC_nottingham','AC_sheffield','AC_liverpool','AC_middlesborough','AC_hounslow','AC_croydon','AC_birmingham','AC_swansea','AC_wrexham','AC_cheadle_revisit','AC_cheadle_imaging','AC_reading_imaging','AC_newcastle_imaging']
BATCH_VALS = ['batch_Batch_b001','batch_Batch_b002','batch_Batch_b003','batch_Batch_b004','batch_Batch_b005','batch_Batch_b006','batch_Batch_b007','batch_Batch_b008','batch_Batch_b009','batch_Batch_b010','batch_Batch_b011','batch_Batch_b012','batch_Batch_b013','batch_Batch_b014','batch_Batch_b015','batch_Batch_b016','batch_Batch_b017','batch_Batch_b018','batch_Batch_b019','batch_Batch_b020','batch_Batch_b021','batch_Batch_b022','batch_Batch_b023','batch_Batch_b024','batch_Batch_b025','batch_Batch_b026','batch_Batch_b027','batch_Batch_b028','batch_Batch_b029','batch_Batch_b030','batch_Batch_b031','batch_Batch_b032','batch_Batch_b033','batch_Batch_b034','batch_Batch_b035','batch_Batch_b036','batch_Batch_b037','batch_Batch_b038','batch_Batch_b039','batch_Batch_b040','batch_Batch_b041','batch_Batch_b042','batch_Batch_b043','batch_Batch_b044','batch_Batch_b045','batch_Batch_b046','batch_Batch_b047','batch_Batch_b048','batch_Batch_b049','batch_Batch_b050','batch_Batch_b051','batch_Batch_b052','batch_Batch_b053','batch_Batch_b054','batch_Batch_b055','batch_Batch_b056','batch_Batch_b057','batch_Batch_b058','batch_Batch_b059','batch_Batch_b060','batch_Batch_b061','batch_Batch_b062','batch_Batch_b063','batch_Batch_b064','batch_Batch_b065','batch_Batch_b066','batch_Batch_b067','batch_Batch_b068','batch_Batch_b069','batch_Batch_b070','batch_Batch_b071','batch_Batch_b072','batch_Batch_b073','batch_Batch_b074','batch_Batch_b075','batch_Batch_b076','batch_Batch_b077','batch_Batch_b078','batch_Batch_b079','batch_Batch_b080','batch_Batch_b081','batch_Batch_b082','batch_Batch_b083','batch_Batch_b084','batch_Batch_b085','batch_Batch_b086','batch_Batch_b087','batch_Batch_b088','batch_Batch_b089','batch_Batch_b090','batch_Batch_b091','batch_Batch_b092','batch_Batch_b093','batch_Batch_b094','batch_Batch_b095','batch_UKBiLEVEAX_b1','batch_UKBiLEVEAX_b10','batch_UKBiLEVEAX_b11','batch_UKBiLEVEAX_b3','batch_UKBiLEVEAX_b4','batch_UKBiLEVEAX_b5','batch_UKBiLEVEAX_b6','batch_UKBiLEVEAX_b7','batch_UKBiLEVEAX_b8','batch_UKBiLEVEAX_b9']
ALL_COVARS = INCLUDED_COVARS[:-2] + AC_VALS + BATCH_VALS
NUM_PC = 40
for i in range(NUM_PC):
    ALL_COVARS.append(f"PC{i+1}")


# phenotype generation constants
TARGET_VARIANCE = 1
DISEASE_PREVELANCE = 0.02
PHENOTYPE_DIR_RESULT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/phenotypes/"
INCLUDE_COVAR = True
AC_EFFECT_SCORE = 1
BATCH_EFFECT_SCORE = 1

CREATE_COVAR_FILES = True # should we save the covar files
BINARY_VARIANT = False


# do a logistic regression between PC and phenotype, to show that snps 
# do not effect through population structure
SEX_EFFECT = 3
YOB_EFFECT = 1
PC_EFFECT_MAX = 0.3

COVAR_GENETIC_EFFECT_RATIO = 2
HERETEBILITY = 0.2
COVAR_EFFECT = 0.5


HERETABILITY_SCORES = [0,0.1,0.2,0.3,0.5,0.7]
COVARIATE_SCORES = [0,0.1,0.2,0.3,0.5,0.7]
# TODO change the name and function to heretibility to heretibility
# heretibility is the variance of the genetics / (var genetcis + var covar)

AC_EFFECT = [AC_EFFECT_SCORE]*len(AC_VALS)
BATCH_EFFECT = [BATCH_EFFECT_SCORE]*len(BATCH_VALS)
# add PC stratagy
PC_EFFECT = list(np.arange(PC_EFFECT_MAX,0,-PC_EFFECT_MAX/NUM_PC))
COVAR_EFFECTS = [SEX_EFFECT,YOB_EFFECT] + AC_EFFECT + BATCH_EFFECT + PC_EFFECT

# TODO: change to work as beta.
# create a function that accepts a vector of Betas and a flag if its binary/continues and add an option for covariates
# If its binary, add prevelance considiration
# for a person J the score for J is the sum over all betas of SNPS * the number of alleles. 
# add a noise so that the variance of phenotype will be 1 (adding noise using 1-sigma) call this Yj
# to do so calculate overall variance after simulation, and add a noise with 1- the variance (make sure that the genetic + covar variance is below 1)
# in continues state, use Yj as phenotype (maybe mul by a constant)
# in binary state the same, need to do a logit (sigmuid and then threashold or just threshold) transformation. detarmine the threshold so that X% of the population are sick
# another way, in the binary state instead of noise can do a bernuli on the result of the sigmuid (a value between 0 and 1)
# the covariats can be treated the same way the betas are used
# read about genetic architucture, compare covariates and betas effect.
# the aggregative effect of snps should be larger, but the effect of a single snp should be smaller.

# look at GWAS catalog, use a table from there for a specific disease and take their betas (find out how to calculate beta from OR)

# variant parameter
NUMBER_OF_MAJOR_VARIANTS = 30
NUMBER_OF_MEDIUM_VARIANTS = 80
NUMBER_OF_LESSER_VARIANTS = 120

MAJOR_VARIANT_EFFECT = 0.05
MEDIUM_VARIANT_EFFECT = 0.015
LESSER_VARIANT_EFFECT = 0.005

# change to betas
MAJOR_VARIANT_DELTA = 1
MEDIUM_VARIANT_DELTA = 0.3
LESSER_VARIANT_DELTA = 0.01


# TODO: implement memory and threads in script
MEMORY = 10000
THREADS = 10

COVAR_GROUPS = [
    "covariates_year_of_birth_0_PC",
    "covariates_sex_0_PC",
    "covariates_sex_year_of_birth_0_PC",
    "covariates_5_PC",
    "covariates_sex_year_of_birth_40_PC",
    "covariates_sex_year_of_birth_batch_40_PC",
    "covariates_sex_year_of_birth_AC_40_PC",
    "covariates_sex_year_of_birth_AC_batch_40_PC"
]


In [5]:
# G = pdp.read_plink1_bin(os.path.join(GENETICS_PATH,TEST_CHROM) + ".bed",os.path.join(GENETICS_PATH,TEST_CHROM) + ".bim",os.path.join(GENETICS_PATH,TEST_CHROM) + ".fam")

In [6]:
# %%time
# G_values = []
# for i in range(1,23):
#     chrom = f"{CHROM_PREFIX}{i}{CHROM_SUFFIX}"
#     print(i)
#     G = pdp.read_plink1_bin(os.path.join(GENETICS_PATH,chrom) + ".bed",
#                             os.path.join(GENETICS_PATH,chrom) + ".bim",
#                             os.path.join(GENETICS_PATH,chrom) + ".fam")
#     G_values.append(G)


In [7]:
covar_df = pd.read_csv(os.path.join(COVARS_PATH,TEST_COVAR),sep="\t",header=None)
covar_df = covar_df.set_axis(["iid","fid"] + ALL_COVARS, axis=1)
legal_iid = set(covar_df["iid"].astype(str))

In [8]:
# sizes = []
# all_variants = []
# k = (NUMBER_OF_MAJOR_VARIANTS + NUMBER_OF_MEDIUM_VARIANTS + NUMBER_OF_LESSER_VARIANTS) * 2
# for G_val in G_values:
#     sizes.append(G_val.shape[1])
# sizes_sum = sum(sizes)
# for i in range(len(G_values)):
#     to_select = int(k * sizes[i] /sizes_sum)
#     rng = np.random.default_rng()
#     selected_indices = rng.choice(sizes[i], size=to_select, replace=False)
#     G_subset = G_values[i].isel(variant=selected_indices)
#     G_subset = G_subset.sel(sample=list(legal_iid.intersection(set(G_subset.sample.values))))
#     G_values[i] = G_subset
#     print(i,G_subset.shape)


In [9]:
# G_combined = xr.concat(G_values, dim="variant")
G_combined = pdp.read_plink1_bin(os.path.join(GENETICS_PATH,TEST_CHROM) + ".bed",
                            os.path.join(GENETICS_PATH,TEST_CHROM) + ".bim",
                            os.path.join(GENETICS_PATH,TEST_CHROM) + ".fam")


Mapping files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:32<00:00, 30.71s/it]


In [12]:
rng = np.random.default_rng(seed=12345)
k = (NUMBER_OF_MAJOR_VARIANTS + NUMBER_OF_MEDIUM_VARIANTS + NUMBER_OF_LESSER_VARIANTS) + 100
selected_indices = rng.choice(G_combined.shape[1], size=k, replace=False)
G_filtered = G_combined.isel(variant=selected_indices)
G_filtered = G_filtered.sel(sample=list(legal_iid.intersection(set(G_filtered.sample.values))))


In [ ]:
# %%time
# dataframes = []
# counter = 0
# for G_val in G_values:
#     print(counter)
#     counter+=1
#     df = G_val.to_dataframe().reset_index()
#     dataframes.append(G_val.to_dataframe())

In [ ]:
# # Select variants
# num_samples, num_variants = G.shape
# k = NUMBER_OF_MAJOR_VARIANTS + NUMBER_OF_MEDIUM_VARIANTS + NUMBER_OF_LESSER_VARIANTS + 100
# rng = np.random.default_rng(seed=12345)
# selected_indices = rng.choice(num_variants, size=k, replace=False)
# print(num_samples)
# G_subset = G.isel(variant=selected_indices)
# G = G_subset
# print(G_subset.shape)


In [ ]:
# variant_ids = G_subset.coords["snp"].values


In [ ]:
# df = G.to_dataframe()
# df = df.reset_index()
# covar_df = pd.read_csv(os.path.join(COVARS_PATH,TEST_COVAR),sep="\t",header=None)
# covar_df = covar_df.set_axis(["iid","fid"] + ALL_COVARS, axis=1)
# legal_iid = set(covar_df["iid"].astype(str))
# df = df[df["iid"].isin(legal_iid)]

In [15]:
def get_unique_variants(variants):
    unique_variants = variants[:]
    random.shuffle(variants)
    return unique_variants

def get_variant_groups(unique_variants):
    major_variants = set(unique_variants[0:NUMBER_OF_MAJOR_VARIANTS])
    medium_variants = set(unique_variants[NUMBER_OF_MAJOR_VARIANTS:NUMBER_OF_MAJOR_VARIANTS+NUMBER_OF_MEDIUM_VARIANTS])
    lesser_variants = set(unique_variants[NUMBER_OF_MAJOR_VARIANTS+NUMBER_OF_MEDIUM_VARIANTS:NUMBER_OF_MAJOR_VARIANTS+NUMBER_OF_MEDIUM_VARIANTS+NUMBER_OF_LESSER_VARIANTS])
    return major_variants, medium_variants, lesser_variants

def create_effect_dict(included_variants,major_variants,medium_variants,lesser_variants):
    effect_dict = {}
    for var in included_variants:
        if var in major_variants:
            effect_dict[var] = MAJOR_VARIANT_DELTA
            continue
        if var in medium_variants:
            effect_dict[var] = MEDIUM_VARIANT_DELTA
            continue
        effect_dict[var] = LESSER_VARIANT_DELTA
    return effect_dict

In [16]:
samples_list = G_filtered.sample.to_series()
snp_list = list(G_filtered.snp.to_series())


In [17]:
len(snp_list)

330

In [18]:

# samples_list = G_combined.sample.to_series()
# G_subset = G.sel(sample=test_ids)
# sample_df = G_subset.to_dataframe()


# sample_df = sample_df.reset_index()
# sample_df = sample_df[sample_df["iid"].isin(legal_iid)]
# print(sample_df)
unique_variants = get_unique_variants(snp_list)
major_variants, medium_variants, lesser_variants = get_variant_groups(unique_variants)
included_variants = major_variants.union(medium_variants).union(lesser_variants)
covar_df['year_of_birth'] = (covar_df['year_of_birth'] - covar_df['year_of_birth'].mean()) / covar_df['year_of_birth'].std()
effect_dict = create_effect_dict(included_variants,major_variants,medium_variants,lesser_variants)
covar_df["covar_effects_chance"] = None
covar_df["genetic_chance"] = None

In [19]:
def get_variant_specific_df(G_subset):
    df = G_subset.to_dataframe()
    df = df.reset_index()
    df = df[df["snp"].isin(included_variants)]
    return df

def get_genotype_dict(df):
    genotype_dict = df[["iid","snp","genotype"]].set_index(["iid","snp"]).to_dict(orient="index")
    for key in genotype_dict.keys():
        genotype_dict[key] = genotype_dict[key]["genotype"]

In [20]:



for i in range(0,len(samples_list),PERSON_BATCH_SIZE):
    if i % 6000 == 0:
        clear_output(wait=True)
        print(i,int( 100*(i /len(samples_list))),"%")

    start = i
    end = i+PERSON_BATCH_SIZE
    test_ids = list(samples_list[start:end])
    G_subset = G_filtered.sel(sample=test_ids)
    df = get_variant_specific_df(G_subset)
    genotype_dict = get_genotype_dict(df)
    # calculate effect for each variant for each person TODO: extract to function
    df["effect"] = df["snp"].map(effect_dict)
    df["effect"] = df["effect"]*df["genotype"]
    df = df.reset_index()

    covar_df["covar_effects_chance"] = ((np.array(covar_df[covar_df.columns[2:-2]]) @ (np.array(COVAR_EFFECTS)))) 
    
    # TODO: change name
    disease_chance_dict = (1 - df.groupby("iid")["effect"].sum()).to_dict()
    # disease_chance_dict
    # TODO: change name, this is not a chance, but a result
    covar_df["genetic_chance"] = covar_df["iid"].astype(str).map(disease_chance_dict).fillna(covar_df["genetic_chance"])
    # adjust so genetic effect and covar effect match the requested


# afterwards


330000 99 %


In [21]:
covar_df = covar_df.dropna()

In [22]:
covar_df["covar_effects_chance"] = (covar_df["covar_effects_chance"]-covar_df["covar_effects_chance"].mean())/covar_df["covar_effects_chance"].std()
covar_df["genetic_chance"] = (covar_df["genetic_chance"]-covar_df["genetic_chance"].mean())/covar_df["genetic_chance"].std()


In [23]:
covar_df.to_csv("temp_covar_df.csv")

In [ ]:
# covar_df = covar_df.set_index("iid")


In [ ]:
assert False

In [24]:
# [SEX_EFFECT,YOB_EFFECT] + AC_EFFECT + BATCH_EFFECT + PC_EFFECT
pc_total_effect = sum(PC_EFFECT)
other_total_effect = sum([SEX_EFFECT,YOB_EFFECT] +[AC_EFFECT_SCORE,BATCH_EFFECT_SCORE])
print("PC effect",pc_total_effect)
print("other covars effect",other_total_effect)
print("PC relative effect in PC",pc_total_effect / (pc_total_effect+other_total_effect))


PC effect 6.149999999999994
other covars effect 6
PC relative effect in PC 0.5061728395061725


In [25]:
# ((np.array(covar_df[covar_df.columns[2:-2]]) @ (np.array(COVAR_EFFECTS))))
# COVAR_EFFECTS = [SEX_EFFECT,YOB_EFFECT] + AC_EFFECT + BATCH_EFFECT + PC_EFFECT
abs_array = np.abs(np.array(covar_df[covar_df.columns[2:-2]]))
abs_scores =  abs_array@np.array(COVAR_EFFECTS)
covar_only_effects = [0,0] + [0]*(len(AC_EFFECT) + len(BATCH_EFFECT)) + PC_EFFECT
covar_only_scores = abs_array@np.array(covar_only_effects)
pc_only_scores = covar_only_scores/abs_scores
print("weighted pc relative effect in PC",np.mean(pc_only_scores))

weighted pc relative effect in PC 0.804002242271349


In [26]:
# print create JSON with parameters
parameters_to_save_for_version = {
    "VERSION":VERSION,
    "TARGET_VARIANCE":TARGET_VARIANCE,
    "DISEASE_PREVELANCE":DISEASE_PREVELANCE,
    "NUMBER_OF_MAJOR_VARIANTS":NUMBER_OF_MAJOR_VARIANTS,
    "NUMBER_OF_MEDIUM_VARIANTS":NUMBER_OF_MEDIUM_VARIANTS,
    "NUMBER_OF_LESSER_VARIANTS":NUMBER_OF_LESSER_VARIANTS,
    "MAJOR_VARIANT_DELTA":MAJOR_VARIANT_DELTA,
    "MEDIUM_VARIANT_DELTA":MEDIUM_VARIANT_DELTA,
    "LESSER_VARIANT_DELTA":LESSER_VARIANT_DELTA,
    "BINARY_VARIANT" : BINARY_VARIANT
}
if INCLUDE_COVAR:

    parameters_to_save_for_version["AC_EFFECT_SCORE"] = AC_EFFECT_SCORE
    parameters_to_save_for_version["BATCH_EFFECT_SCORE"] = BATCH_EFFECT_SCORE
    parameters_to_save_for_version["SEX_EFFECT"] = SEX_EFFECT
    parameters_to_save_for_version["YOB_EFFECT"] = YOB_EFFECT
    parameters_to_save_for_version["PC_EFFECT_MAX"] = PC_EFFECT_MAX
json_file_name = VERSION + "_parameters" + ".json"
if os.path.exists(json_file_name):
    raise Exception("Version file exists")
with open(json_file_name, 'w', encoding='utf-8') as f:
    json.dump(parameters_to_save_for_version, f, ensure_ascii=False, indent=4)

In [27]:
# TODO: add creation of multiple phenotype files
covar_dfs = []
for her_score in HERETABILITY_SCORES:
    for cov_score in COVARIATE_SCORES:
        print(her_score,cov_score)
        temp_cov_df = covar_df.copy()
        gene_var = ((temp_cov_df["genetic_chance"]).var())
        covar_var = (temp_cov_df["covar_effects_chance"].var())
        genetic_factor = np.sqrt(her_score/gene_var)
        covar_factor = np.sqrt(cov_score/covar_var)
        temp_cov_df["genetic_chance"] = temp_cov_df["genetic_chance"] * genetic_factor
        temp_cov_df["covar_effects_chance"] = temp_cov_df["covar_effects_chance"] * covar_factor
        temp_cov_df["disease_chance"] = temp_cov_df["genetic_chance"] + temp_cov_df["covar_effects_chance"]
        added_variance = TARGET_VARIANCE -temp_cov_df["disease_chance"].var()
        # input(added_variance)
        mu, sigma = 0, added_variance # mean and standard deviation
        noise = np.random.normal(mu, sigma**0.5, len(temp_cov_df))
        temp_cov_df["disease_chance"] = temp_cov_df["disease_chance"] + noise
        # temp_cov_df["disease_chance"].var()
        quantile = (temp_cov_df[["disease_chance"]].quantile(1 - DISEASE_PREVELANCE, method="table", interpolation="nearest")["disease_chance"])
        if BINARY_VARIANT:
            temp_cov_df['disease_indicator'] = (
                temp_cov_df['disease_chance'] > quantile
            ).astype(int).astype(float)
        else:
            temp_cov_df['disease_indicator'] = temp_cov_df['disease_chance']
        # temp_cov_df.dropna()
        covar_dfs.append([temp_cov_df,her_score,cov_score])
        if CREATE_COVAR_FILES:
            temp_cov_df.to_csv(f"{COVAR_DF_PREFIX}_v={VERSION}_her={her_score}_cov={cov_score}.csv")
        

0 0
0 0.1
0 0.2
0 0.3
0 0.5
0 0.7
0.1 0
0.1 0.1
0.1 0.2
0.1 0.3
0.1 0.5
0.1 0.7
0.2 0
0.2 0.1
0.2 0.2
0.2 0.3
0.2 0.5
0.2 0.7
0.3 0
0.3 0.1
0.3 0.2
0.3 0.3
0.3 0.5
0.3 0.7
0.5 0
0.5 0.1
0.5 0.2
0.5 0.3
0.5 0.5
0.5 0.7


/tmp/ipykernel_3155768/2502292744.py:17: RuntimeWarning: invalid value encountered in scalar power
  noise = np.random.normal(mu, sigma**0.5, len(temp_cov_df))


0.7 0
0.7 0.1
0.7 0.2
0.7 0.3
0.7 0.5


/tmp/ipykernel_3155768/2502292744.py:17: RuntimeWarning: invalid value encountered in scalar power
  noise = np.random.normal(mu, sigma**0.5, len(temp_cov_df))


0.7 0.7


/tmp/ipykernel_3155768/2502292744.py:17: RuntimeWarning: invalid value encountered in scalar power
  noise = np.random.normal(mu, sigma**0.5, len(temp_cov_df))


In [28]:
i = 1
print(covar_dfs[i][0]["disease_indicator"].dropna().mean())
print("her_score",covar_dfs[i][1])
print("cov_score",covar_dfs[i][2])

-0.0018202619673667745
her_score 0
cov_score 0.1


In [29]:
covar_dfs[i][0]

,iid,fid,sex,year_of_birth,AC_stockport_pilot,AC_manchester,AC_oxford,AC_cardiff,AC_glasgow,AC_edinburgh,...,PC35,PC36,PC37,PC38,PC39,PC40,covar_effects_chance,genetic_chance,disease_chance,disease_indicator
0,5895328,5895328,1.0,-1.152223,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.419695,-1.412190,5.011600,-2.041770,-0.329472,-0.610231,-0.459752,0.0,-1.360605,-1.360605
1,5764375,5764375,0.0,-0.024885,0.0,0.0,1.0,0.0,0.0,0.0,...,-3.076080,0.282474,1.712010,-1.723410,-0.140920,1.834630,-0.202652,0.0,0.259322,0.259322
2,5503073,5503073,1.0,-0.901703,0.0,0.0,0.0,1.0,0.0,0.0,...,6.917620,3.797780,-0.783315,-1.714340,4.746740,-0.425447,0.351544,0.0,-0.007729,-0.007729
3,2563626,2563626,1.0,-0.150145,0.0,0.0,0.0,0.0,0.0,0.0,...,0.180031,0.908227,-1.488970,-2.507100,-2.025750,7.023560,-0.114147,-0.0,-1.610841,-1.610841
4,4980832,4980832,1.0,-0.400665,0.0,0.0,0.0,0.0,0.0,0.0,...,3.000040,1.880090,0.433608,-2.291620,-2.461820,-3.445180,0.153879,0.0,0.240332,0.240332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333366,2243883,2243883,1.0,-0.275405,0.0,1.0,0.0,0.0,0.0,0.0,...,-1.756630,-5.489920,2.986760,-0.533189,-0.582788,0.070181,0.006754,0.0,0.963406,0.963406
333367,3570729,3570729,1.0,0.476153,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.785830,1.074660,-2.581080,-0.671792,2.334690,-3.509470,0.393571,0.0,-0.649838,-0.649838
333368,1663456,1663456,1.0,-1.277482,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.008020,-1.370330,-3.174280,-3.719140,-1.817060,0.452773,0.212045,0.0,0.182012,0.182012
333369,3118989,3118989,0.0,-1.152223,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.979120,4.153130,-5.280610,-2.201190,-0.167185,-0.494174,0.102179,-0.0,0.638484,0.638484


In [33]:
variant_dict = {}

variant_dict["major_vars"] = list(major_variants)
variant_dict["med_vars"] = list(medium_variants)
variant_dict["lesser_vars"] = list(lesser_variants)
json_file_name = VERSION + "_variants.json"
with open(json_file_name, 'w', encoding='utf-8') as f:
    json.dump(variant_dict, f, ensure_ascii=False, indent=4)

In [31]:
commands = []
for temp_cov_df,her_score,cov_score in covar_dfs:
    phenotype_file_result_name = (
        f"generated_pehn_result_ver={VERSION}_heretability={her_score}_covar_effect={cov_score}")
    phenotype_file_result_name = phenotype_file_result_name+".txt"
    phenotype_file_path = os.path.join(PHENOTYPE_DIR_RESULT_PATH,phenotype_file_result_name)
    temp_cov_df[["iid","fid","disease_indicator"]].to_csv(phenotype_file_path,index=False,header=False,sep="\t")

    chrom_path = os.path.join(GENETICS_PATH,TEST_CHROM)
    result_path = os.path.join(PLINK_RESULT_DIR,phenotype_file_result_name.replace(".txt",""))
    for covar_type in COVAR_GROUPS:
        covar_file = os.path.join(COVARS_PATH,covar_type + ".txt")
        result_path = os.path.join(PLINK_RESULT_DIR,phenotype_file_result_name.replace(".txt",covar_type))
        command =f"{PLINK_PATH} --bed {chrom_path}.bed --bim {chrom_path}.bim --fam {chrom_path}.fam --pheno {phenotype_file_path} --covar {covar_file}  --out  {result_path} --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000 --1 --adjust"
        # print(command)
        # print(f'sbatch --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{command}"')
        commands.append(f'sbatch --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{command}"')
    command =f"{PLINK_PATH} --bed {chrom_path}.bed --bim {chrom_path}.bim --fam {chrom_path}.fam --pheno {phenotype_file_path} --out  {result_path} --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000 --1  --adjust"
    # print(command)
    result_path = os.path.join(PLINK_RESULT_DIR,phenotype_file_result_name.replace(".txt","_no_covar"))
    # print(f'sbatch --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{command}"')
    commands.append(f'sbatch --mem=12g -c10 --time=3-0 --requeue --killable --wrap="{command}"')
    

In [32]:
temp_file = "temp_file.sh"
counter = 0
with open(temp_file,"w") as f:
    for com in commands:
        f.write(com + "\n")
        f.write("sleep 2\n")
        counter+=1
        if counter%30 == 0:
            f.write("sleep 10\n")

!chmod 744 {temp_file}


In [ ]:
# included_variants = major_variants.union(medium_variants).union(lesser_variants)
# # TODO: add protective variants
assert False

In [ ]:
# df = df[df["variant"].isin(included_variants)]

## GWAS simulation visualisation

In [39]:
PVAL_THRESHOLD = 5*10**-3

In [35]:
json_file_name = VERSION + "_variants.json"
with open(json_file_name) as json_file:
        variant_data = json.load(json_file)


In [37]:
# var_dict = sample_df[["variant","snp"]].set_index("variant").to_dict()["snp"]


In [ ]:
# for key in variant_data:
#     for i in range(len(variant_data[key])):
#         variant_data[key][i] = var_dict[variant_data[key][i]]

In [42]:
results = []
for her_score in HERETABILITY_SCORES:
    for cov_score in COVARIATE_SCORES:
        if her_score + cov_score > 1:
            continue
        for covar_type in COVAR_GROUPS:
            try:
                phenotype_file_result_name = (
                    f"generated_pehn_result_ver={VERSION}_heretability={her_score}_covar_effect={cov_score}")
                phenotype_file_result_name = phenotype_file_result_name+".txt"
                result_path = os.path.join(PLINK_RESULT_DIR,phenotype_file_result_name.replace(".txt",covar_type))
                covar_file = os.path.join(COVARS_PATH,covar_type + ".txt")
                result_path = os.path.join(PLINK_RESULT_DIR,phenotype_file_result_name.replace(".txt",covar_type) + F".{FILE_ENDING}")
                df = pd.read_csv(result_path,sep="\t")
                df = df[df["P"] < PVAL_THRESHOLD]
                variants = set(df["ID"])
                found_major = variants.intersection( variant_data["major_vars"])
                found_medium = variants.intersection( variant_data["med_vars"])
                found_lesser = variants.intersection( variant_data["lesser_vars"])
                if VERBOSE:
                    print(covar_type,her_score,cov_score)
                    print("found_major",len(found_major),"/",len(variant_data["major_vars"]))
                    print("found_medium",len(found_medium),"/",len(variant_data["med_vars"]))
                    print("found_lesser",len(found_lesser),"/",len(variant_data["lesser_vars"]))
                    print("found_overall", len(variants))
                    print("found_extra",len(variants - found_major.union(found_medium).union(found_lesser)))
                results.append(
                    [her_score,cov_score,covar_type,len(found_major),len(found_medium),
                     len(found_lesser),len(variants),len(variants - found_major.union(found_medium).union(found_lesser))])
                # print(variants)
            except OSError as exc:
                print(exc)
                print(covar_file,"failed")


In [44]:
cols = ["her_score","cov_score","covar_type","found_major","found_medium","found_lesser","found_overall","found_extra"]
res_df = pd.DataFrame(results,columns=cols)


In [51]:
# for her_score in HERETABILITY_SCORES:
#     for cov_score in COVARIATE_SCORES:
#         print(her_score,cov_score,res_df[(res_df["her_score"] == her_score)
#               & (res_df["cov_score"] == cov_score)]["found_extra"].mean())

0 0 9.0
0 0.1 41.125
0 0.2 75.0
0 0.3 103.75
0 0.5 149.625
0 0.7 184.625
0.1 0 11.0
0.1 0.1 42.25
0.1 0.2 70.125
0.1 0.3 96.75
0.1 0.5 135.25
0.1 0.7 184.25
0.2 0 19.875
0.2 0.1 41.75
0.2 0.2 75.0
0.2 0.3 100.25
0.2 0.5 148.75
0.2 0.7 180.25
0.3 0 9.75
0.3 0.1 39.5
0.3 0.2 80.5
0.3 0.3 104.875
0.3 0.5 147.0
0.3 0.7 187.5
0.5 0 16.25
0.5 0.1 42.25
0.5 0.2 75.0
0.5 0.3 94.25
0.5 0.5 154.0
0.5 0.7 nan
0.7 0 15.5
0.7 0.1 44.25
0.7 0.2 74.75
0.7 0.3 101.25
0.7 0.5 nan
0.7 0.7 nan
